# Лаборатория биокибернетики

Исследователи из лаборатории биокибернетики разработали алгоритм для классификации биологических объектов.

Однажды младший научный сотрудник предложил простое определение: «Объект класса X — это организм с двумя конечностями и без перьевого покрова». Но старший коллега, известный своим остроумием, принёс в лабораторию ощипанную курицу и заявил: «По вашему определению, это тоже объект X!». Пришлось уточнить критерий: «…и с плоскими когтями».

Чтобы автоматизировать классификацию, исследователи закодировали признаки организмов латинскими буквами от A до I. Они не раскрыли, что означает каждый признак, поэтому их интерпретация остаётся неизвестной.

Позже команда собрала данные:

- Младший сотрудник подготовил обучающую выборку с метками
- Старший — тестовую выборку, но забыл указать классы объектов и ушёл на перерыв

Задача: Используя обучающие данные, предскажите метки классов для тестовой выборки.

## О датасете

- Файл `train.csv` содержит признаки обучающей выборки и колонку с разметкой `target`. Значение 1 в этой колонке соответствует человеку, а 0 – нечеловеку.
- Файл `test.csv` содержит признаки тестовой выборки.
- Файл `example.csv` содержит пример корректной посылки в контест

Таким образом, вам нужно предсказать колонку `target` для объектов из файла `test.csv`.

## Что нужно сделать

От вас требуется загрузить в систему файл `answers.csv` в формате, аналогичном файлу `example.csv` с предсказаниями для объектов тестовой выборки. В качестве целевой метрики используется [ROC-AUC](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc). Баллы за это задание рассчитываются по формуле: $100∗max(min((AUC−0.8)/0.08,1),0)$, где AUC - значение ROC-AUC ваших предсказаний. Таким образом, для максимального числа баллов необходимо набрать $AUC≥0.88$.

## Решение

In [1]:
import pandas as pd

# Посмотрим на данные в примере
pd.read_csv("../data/02/example.csv").head()

,target
0,0
1,1
2,1
3,0
4,0


In [5]:
# Посмотрим на данные в тренировочном датасете
train_path = "../data/02/train.csv"
df = pd.read_csv(train_path)
df.head()

,A,B,C,D,E,F,G,H,I,target
0,0.505,8,-,1.984,3.0,5,2.642,-5.122,0.649,1
1,0.536,4,-,1.977,1.0,3,5.756,-3.077,0.950,0
2,0.024,3,-,3.147,2.0,6,2.435,4.387,2.186,1
3,0.543,4,-,2.440,3.0,9,4.440,7.730,1.938,0
4,0.942,8,-,1.952,3.0,9,7.176,-4.579,0.346,1


In [3]:
df.describe()

,A,B,D,E,F,G,H,I,target
count,1000.000000,1000.000000,1000.000000,925.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.504627,4.940000,2.038728,2.036757,5.578000,5.064382,-0.156917,1.197345,0.288000
std,0.317773,2.283772,0.346977,0.826214,2.877185,1.600964,5.329461,0.770921,0.453058
min,0.000000,0.000000,1.649000,1.000000,1.000000,0.735000,-10.750000,-0.125000,0.000000
25%,0.218750,3.000000,1.783000,1.000000,3.000000,3.674750,-5.031750,0.546500,0.000000
50%,0.511000,5.000000,1.944000,2.000000,6.000000,5.483000,-1.220500,1.103000,0.000000
75%,0.806500,6.000000,2.201250,3.000000,8.000000,6.249500,4.925000,1.644250,1.000000
max,1.000000,15.000000,3.771000,3.000000,10.000000,8.375000,11.475000,4.279000,1.000000


### Что видим в данных

- В признаке С значения `-`
- Судя по count = 925 у E, там есть 75 пропусков. Нужно будет либо заполнить, например, медианой, либо использовать модели, которые умеют работать с NaN (HistGradientBoostingClassifier, XGBoost, CatBoost).
- Среднее target = 0.288. Класс `1` встречается примерно в 29% случаев, значит, классы несбалансированы. Лучше использовать ROC-AUC (что и задано в условии), а не accuracy для оценки.
- Разнородные масштабы признаков. Некоторые признаки от 0 до 1, другие – от -10 до +10. Для линейных моделей (например, логистической регрессии) желательно нормализовать, для деревьев – не критично.

### Что будем делать

Это классическая бинарная классификация с лёгким дисбалансом классов, разнородными признаками и небольшим числом наблюдений (1000). По сути, нужно научиться различать «человека» и «не человека» по девяти числовым признакам.

### Пробуем логистическую регрессию в качестве бейзлайна

Для начала попробуем решить проблему с пропусками.

In [6]:
import numpy as np

def clean_numeric(df, cols=None):
    # по умолчанию чистим все признаки
    if cols is None:
        cols = [c for c in df.columns if c != "target"]
    
    # заменяем странные пропуски на NaN
    df = df.copy()
    df[cols] = df[cols].replace(
        {
            "": np.nan,
            "-": np.nan,
            "–": np.nan,
            "—": np.nan,
            "NA": np.nan,
            "na": np.nan,
            "NaN": np.nan,
        }
    )
    
    # приводим к числам, всё неподдающееся -> NaN
    for c in cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    
    return df

# загрузка
train = pd.read_csv("../data/02/train.csv")
test  = pd.read_csv("../data/02/test.csv")

# очистка
train = clean_numeric(train)
test  = clean_numeric(test, cols=test.columns)

X = train.drop(columns=["target"])
y = train["target"]

In [8]:
# Снова проверим пропуски
print(X.isna().sum())

A       0
B       0
C    1000
D       0
E      75
F       0
G       0
H       0
I       0
dtype: int64


In [9]:
X["C"].isna().sum(), len(X)

(np.int64(1000), 1000)

In [10]:
X["E"].isna().sum(), len(X)

(np.int64(75), 1000)

In [17]:
# Удаляем полностью пустые колонки, в нашем случае это будет только C
X = X.dropna(axis=1, how="all")

In [18]:
from sklearn.model_selection import train_test_split

# Делим на train/val
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [19]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Создаём pipeline: заполнение пропусков + стандартизация + логистическая регрессия
pipe = make_pipeline(
    SimpleImputer(strategy="median"),
    StandardScaler(),
    LogisticRegression(max_iter=500, class_weight="balanced", random_state=42)
)

In [20]:
# Тренируем модель
pipe.fit(X_train, y_train)

,steps,"[('simpleimputer', ...), ('standardscaler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'median'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,copy,True


In [22]:
from sklearn.metrics import roc_auc_score

# Валидируем
proba_val = pipe.predict_proba(X_val)[:, 1]
auc = roc_auc_score(y_val, proba_val)
score = 100 * max(min((auc - 0.8) / 0.08, 1), 0)

print(f"ROC-AUC = {auc:.4f} | Score = {score:.2f}")

ROC-AUC = 0.7995 | Score = 0.00


In [24]:
# Сохраняем результат
test_clean = test.dropna(axis=1, how="all")
proba_test = pipe.predict_proba(test_clean)[:, 1]
pd.DataFrame({"id": range(len(proba_test)), "target": proba_test}).to_csv("02-answers.csv", index=False)

In [25]:
pd.read_csv("02-answers.csv").head()

,id,target
0,0,0.617267
1,1,0.678942
2,2,0.355036
3,3,0.305649
4,4,0.037546


### Логистическая регрессия не подошла

- Слишком простой baseline. Данные, похоже, нелинейные.
- Фичи грязные или не совпадают между train/test
- Разная распределённость train/test

In [26]:
test = test[X.columns]

In [43]:
# пробуем бустинг
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

param_grid = {
    "learning_rate": [0.03, 0.05, 0.08, 0.1],
    "max_depth": [4, 6, 8],
    "max_leaf_nodes": [31, 63],
    "min_samples_leaf": [10, 20, 30],
}

clf = HistGradientBoostingClassifier(random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(clf, param_grid, scoring="roc_auc", cv=cv, n_jobs=-1)
grid.fit(X, y)

print("Best params:", grid.best_params_)
print("Best ROC-AUC:", grid.best_score_)

Best params: {'learning_rate': 0.08, 'max_depth': 6, 'max_leaf_nodes': 31, 'min_samples_leaf': 30}
Best ROC-AUC: 0.8768066340584937


In [45]:
clf = HistGradientBoostingClassifier(
    **grid.best_params_,
    random_state=42
)

clf

,loss,'log_loss'
,learning_rate,0.08
,max_iter,100
,max_leaf_nodes,31
,max_depth,6
,min_samples_leaf,30
,l2_regularization,0.0
,max_features,1.0
,max_bins,255
,categorical_features,'from_dtype'
,monotonic_cst,None


In [46]:
clf.fit(X, y)
proba_test = clf.predict_proba(test)[:, 1]

In [47]:
sub = pd.DataFrame({
    "id": test.index if "id" not in test.columns else test["id"],
    "target": proba_test
})

In [48]:
sub.to_csv("02-answers.csv", index=False)

In [49]:
pd.read_csv("02-answers.csv").head()

,id,target
0,0,0.042806
1,1,0.035008
2,2,0.011710
3,3,0.009417
4,4,0.014149
